A partir de la información de cada año contenida en el dataset se debe informar, año tras año, el porcentaje de personas mayores a 6 años capaces e incapaces de leer y escribir.
Importante: tomar la información del último trimestre de cada año.

Tabla individual  
ANO4 N (4) Año de relevamiento (4 dígitos)

TRIMESTRE N (1) Ventana de observación  
- = 1° trimestre  
- = 2° trimestre  
- = 3° trimestre  
- = 4° trimestre  

PONDERA N (6) Ponderación  

CH05 date Fecha de nacimiento
(día, mes y año) --> ver como esta dado el dato

CH06 N (2) ¿Cuántos años cumplidos tiene?

CH09 N (1) ¿Sabe leer y escribir?
1 = Sí
2 = No
3 = Menor de 2 años

In [ ]:
def filtrar_trimestre(filas, col_trim=1, trimestre = '4'):
    """Filtra las filas correspondientes al trimestre pasado por parametro como string (valor defecto '4')"""
    return [fila for fila in filas if fila[col_trim] == trimestre]

def agrupar_por_anio(filas, col_anio=0):
    """Agrupa las filas por año (usando índice de columna)"""
    grupos = {}
    for fila in filas:
        anio = fila[col_anio]
        if anio not in grupos:
            grupos[anio] = []
        grupos[anio].append(fila)
    return grupos

def suma_ponderada(filas, condicion, col_pondera=2):
    """Suma ponderada de las filas que cumplen cierta condición"""
    return sum(float(f[col_pondera]) for f in filas if condicion(f))


def analizar_alfabetismo_por_anio(filas, col_anio=0, col_trim=1, col_pondera=2, col_edad=3, col_lee=4):
    """Analiza el alfabetismo por año considerando personas mayores de 6 años"""
    filas_filtradas = filtrar_trimestre(filas, col_trim)
    datos_por_anio = agrupar_por_anio(filas_filtradas, col_anio)

    for anio, filas_anio in datos_por_anio.items():
        total_mayores_6 = suma_ponderada(filas_anio, lambda f: int(f[col_edad]) > 6, col_pondera)
        saben_leer = suma_ponderada(filas_anio, lambda f: int(f[col_edad]) > 6 and f[col_lee] == '1', col_pondera)
        no_saben_leer = suma_ponderada(filas_anio, lambda f: int(f[col_edad]) > 6 and f[col_lee] == '2', col_pondera)

        porcentaje_si = (saben_leer / total_mayores_6) * 100
        porcentaje_no = (no_saben_leer / total_mayores_6) * 100

        print(f"Año {anio}: {porcentaje_si:.2f}% saben leer y escribir, {porcentaje_no:.2f}% no saben.")



# analizar_alfabetismo_por_anio(filas_modificadas)

<span style="color:red;"> el aviso "Importante: tomar la información del último trimestre de cada año." Es solo para el punto 1 o para todos?.</span>

2. A partir de un año y trimestre elegido por el usuario informar el porcentaje de personas no nacidas en Argentina que hayan cursado un nivel universitario o superior.

Tabla individual  
ANO4 N (4) Año de relevamiento (4 dígitos)

TRIMESTRE N (1) Ventana de observación  
- = 1° trimestre  
- = 2° trimestre  
- = 3° trimestre  
- = 4° trimestre  

PONDERA N (6) Ponderación  

CH12 N (2) ¿Cuál es el nivel más alto que cursa o cursó?
- 1 = Jardín/preescolar
- 2 = Primario
- 3 = EGB
- 4 = Secundario
- 5 = Polimodal
- 6 = Terciario
- 7 = Universitario
- 8 = Posgrado universitario
- 9 = Educación especial (discapacitado)

CH15 N (1) ¿Dónde nació?
1. En esta localidad
2. En otra localidad de esta provincia
3. En otra provincia (especificar)
4. En un país limítrofe (especificar: Brasil, Bolivia, Chile, Paraguay, Uruguay)
5. En otro país (especificar)
9. Ns/Nr

NIVEL_ED N (1) Nivel educativo
- 1 = Primario incompleto (incluye educación especial)
- 2 = Primario completo
- 3 = Secundario incompleto
- 4 = Secundario completo
- 5 = Superior universitario incompleto
- 6 = Superior universitario completo
- 7 = Sin instrucción
- 9 = Ns/Nr

<span style="color:red;"> 
creo que para esta consulta deberia considerarse tambien una edad minima   

el procentaje que me pide es respecto a la poblacion total de argentina o respecto a la poblacion total de no nacidos en argentina (hice el porcentaje respecto al total de no nacidos en argentina)
</span>

In [ ]:
def filtrar_anio(filas, col_anio=1, anio = '4'):
    """Filtra las filas correspondientes al trimestre pasado por parametro como string (valor defecto '4')"""
    return [fila for fila in filas if fila[col_anio] == anio]

def es_extranjero(valor_ch15):
    """Devuelve True si la persona nació fuera de Argentina"""
    return valor_ch15 in ['4', '5']

def es_universitario(valor_ch12):
    """Devuelve True si la persona cursó/cursa universidad o posgrado"""
    return valor_ch12 in ['7', '8']

In [ ]:
def analizar_universitarios_extranjeros(filas, anio, trimestre):
    """Calcula el porcentaje de personas no nacidas en Argentina con nivel universitario o superior"""
    
    filas_anio = filtrar_anio(filas, anio=anio)
    filas_trim = filtrar_trimestre(filas_anio, trimestre=trimestre)
    
    total_ponderado_extranjeros = 0
    total_ponderado_universitarios_extranjeros = 0

    for fila in filas_trim:
        ch15 = fila[5]  # lugar de nacimiento
        ch12 = fila[4]  # nivel educativo
        pondera = int(fila[3])  # ponderación

        if es_extranjero(ch15):
            total_ponderado_extranjeros += pondera
            if es_universitario(ch12):
                total_ponderado_universitarios_extranjeros += pondera

    if total_ponderado_extranjeros == 0:
        return 0  # evitar división por cero

    porcentaje = (total_ponderado_universitarios_extranjeros / total_ponderado_extranjeros) * 100
    return round(porcentaje, 2)

<span style="color:red;"> 
en las preguntas que pide seleccionar algun dato ejemplo algun aglomerado. esta seleccion se hace por afuera de la funcion con un input y despues se ingresa a la funcion con esta valor de parametro o pongo el input directamente adentro de la funcion   

por ejemplo en la pregunta 9 "Pedir al usuario que seleccione un aglomerado y a partir de la información contenida retornar una tabla que contenga la cantidad de personas mayores de edad según su nivel de estudios alcanzados."

Lo mejor es que el input() esté afuera de la función.  Esto te permite que la función sea reutilizable, testeable, y más limpia.
</span>